# JEPPESEN PROJECT

### Parse data from NAVDATA_POINTS, NAVDATA_SEGMENTS and the .srad-file from Jeppesen
---
#### Creates the following variables:
* **segments** - list with elements: 
    * `{'airway': 'AWY M771', 'from': 'WPT 34BC', 'to': 'WPT DONDA'}`
* **possible_destinations** - dictionary with keys: 
    * `'WPT ABTUB': {'NDB DP', ... , 'WPT UDINO'}`
* **allowed_routes** - list of lists containing segments: 
    * `[{'from': 'WPT INTIK', 'to': 'WPT ASURI'}, ... , {'from': 'VOR NLG', 'to': 'WPT TAMOT'}]`
* **points** - dictionary with keys:
    * `'WPT 0000E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E000000'}`

In [1]:
from graph import Graph

point_path = "../data/NAVDATA_POINTS"
segment_path = "../data/NAVDATA_SEGMENTS"
srad_path = "../data/_CHINA_MANDATE_OVERFLIGHT.srad"

mapping = {"1": "WPT",
           "2": "VOR",
           "3": "NDB",
           "4": "DME",
           "5": "APT",
           "6": "RWY"}

pointType = {'V': 'VOR',
             'W': 'WPT',
             'N': 'NDB',
             'A': 'APT',
             'D': 'DME',
             'R': 'RWY'}

def readFileLines(path):
    with open(path,'r') as f:
        return f.readlines()
    
def readFile(path):
    with open(path,'r') as f:
        return f.read()

#------------------------------------------------#
#      Create list of all segments in China      #
#------------------------------------------------#
segment_lines = readFileLines(segment_path)
segments = []
for line in segment_lines:
    if line[15]=="Z" or line[29]=="Z":  # pick the segments with at least one node in China ("z")
        segments.append({
            'airway': 'AWY '+line[2:8].strip(),
            'from': mapping[line[21]]+" "+line[9:14].strip(), 
            'to': mapping[line[35]]+" "+line[23:28].strip()
        })
        
#---------------------------------------------------------------------------#
#      Create dictionary with all possible destinations for each point      #
#---------------------------------------------------------------------------#
tuples = [(seg['from'], seg['to']) for seg in segments]
possible_destinations = Graph(tuples, directed=True)._graph

#-------------------------------------------------#
#      Create a list with all allowed routes      #
#-------------------------------------------------#
allowed_routes = []
sequences = readFile(srad_path)
# Clean input:   | indentation     | ")) " before next "(Seq"                              | the first ~30 rows
sequences = [seq.replace('   ','').replace(')) ',')') for seq in sequences.split('(Seq\n')][1:]
sequences = [seq.split('\n')[:-1] for seq in sequences]  # remove empty "" element from each sequence
for seq in sequences:
    tmp_seq = []
    for seg in seq:
        cols = seg.split(' ')
        tmp_seq.append({
            'from': cols[8]+" "+cols[9],
            'to': cols[12]+" "+cols[13]
        })
    allowed_routes.append(tmp_seq)
    
#-----------------------------------------------#
#      Create a dictionary with all points      #
#-----------------------------------------------#
pointlines = readFileLines(point_path)
points = {}
for line in pointlines:
    points[pointType[line[0]]+" "+line[2:7].strip()] = {
        'area': line[19:23].strip(), 
        'lat': line[34:40].strip(), 
        'lon': line[41:48]
    }

In [4]:


entry_nodes = []
for route in allowed_routes:
    if not(route[0]['from'] in entry_nodes):
        entry_nodes.append(route[0]['from'])
        
for entry_node in entry_nodes:
    current_routes = []
    for route in allowed_routes:
        if route[0]['from'] == entry_node:
            current_routes.append(route)
            
    for route in current_routes:
        for node in allowed_routes:
            

        

['WPT INTIK', 'WPT SADLI', 'WPT LINSO', 'WPT MORIT', 'WPT KAMUD', 'WPT MAGIT', 'WPT BEKOL', 'WPT RULAD', 'WPT POLHO', 'WPT SIMLI', 'WPT GOPTO', 'WPT SAGAG', 'WPT PURPA', 'WPT TEBAK', 'WPT AGAVO', 'WPT SIKOU', 'WPT TEBUS', 'WPT NIXAL', 'WPT TOMUK', 'WPT SARIN', 'WPT TELOK', 'WPT BISUN', 'WPT VASRO', 'WPT KATBO', 'VOR ZUH', 'WPT MAGOG', 'WPT ASSAD', 'WPT GOLOT']


In [6]:
allowed_routes

[[{'from': 'WPT INTIK', 'to': 'WPT ASURI'},
  {'from': 'WPT ASURI', 'to': 'WPT PADSA'},
  {'from': 'WPT PADSA', 'to': 'VOR HET'},
  {'from': 'VOR HET', 'to': 'WPT DOTOS'},
  {'from': 'WPT DOTOS', 'to': 'WPT ISBAM'},
  {'from': 'WPT ISBAM', 'to': 'WPT ATPOK'},
  {'from': 'WPT ATPOK', 'to': 'WPT TODAM'},
  {'from': 'WPT TODAM', 'to': 'WPT EKETA'},
  {'from': 'WPT EKETA', 'to': 'VOR TYN'},
  {'from': 'VOR TYN', 'to': 'WPT XIVEP'},
  {'from': 'WPT XIVEP', 'to': 'WPT ANPIG'},
  {'from': 'WPT ANPIG', 'to': 'WPT EGEBI'},
  {'from': 'WPT EGEBI', 'to': 'NDB SQ'},
  {'from': 'NDB SQ', 'to': 'WPT PADNO'},
  {'from': 'WPT PADNO', 'to': 'WPT TAMIX'},
  {'from': 'WPT TAMIX', 'to': 'WPT NOPIN'},
  {'from': 'WPT NOPIN', 'to': 'WPT DUBAG'},
  {'from': 'WPT DUBAG', 'to': 'VOR CGO'},
  {'from': 'VOR CGO', 'to': 'WPT KAMDA'},
  {'from': 'WPT KAMDA', 'to': 'WPT OBLIK'},
  {'from': 'WPT OBLIK', 'to': 'WPT ESDOS'},
  {'from': 'WPT ESDOS', 'to': 'VOR HOK'},
  {'from': 'VOR HOK', 'to': 'VOR LKO'},
  {'from': '

In [23]:
possible_destinations

defaultdict(set,
            {'WPT 34BC': {'WPT DONDA'},
             'VOR ABK': {'WPT GINOM'},
             'WPT ABTUB': {'NDB DP', 'WPT DALIM', 'WPT UDINO'},
             'WPT ABVAX': {'VOR HLD', 'WPT ELPUN'},
             'WPT ABVER': {'VOR XDX', 'WPT LATUX', 'WPT URBAD'},
             'WPT ABVIL': {'WPT ELNEX', 'WPT IDBIN'},
             'WPT ADBAG': {'VOR LXI', 'WPT KATBO', 'WPT MEPAN'},
             'WPT ADBAS': {'VOR HGH', 'WPT KAKIS'},
             'WPT ADBIN': {'WPT BOKAT', 'WPT TOMUD'},
             'WPT ADGOL': {'WPT IKUBA', 'WPT KIKEG'},
             'WPT ADKUN': {'WPT URDIN', 'WPT XELOV'},
             'WPT ADMUX': {'WPT NOLEP', 'WPT TUSLI'},
             'WPT ADNEN': {'NDB HO', 'VOR QIY', 'WPT TEBIB'},
             'WPT ADPET': {'VOR FKG', 'VOR QTV', 'WPT DAKPA', 'WPT NIRAV'},
             'WPT AGANU': {'WPT AMUTA'},
             'WPT AGAVO': {'WPT DONVO', 'WPT GONAV', 'WPT MUDAL', 'WPT NONOS'},
             'WPT AGOXA': {'VOR FKG', 'VOR QTV'},
             'WPT AGPOR': {

In [24]:
allowed_routes

[[{'from': 'WPT INTIK', 'to': 'WPT ASURI'},
  {'from': 'WPT ASURI', 'to': 'WPT PADSA'},
  {'from': 'WPT PADSA', 'to': 'VOR HET'},
  {'from': 'VOR HET', 'to': 'WPT DOTOS'},
  {'from': 'WPT DOTOS', 'to': 'WPT ISBAM'},
  {'from': 'WPT ISBAM', 'to': 'WPT ATPOK'},
  {'from': 'WPT ATPOK', 'to': 'WPT TODAM'},
  {'from': 'WPT TODAM', 'to': 'WPT EKETA'},
  {'from': 'WPT EKETA', 'to': 'VOR TYN'},
  {'from': 'VOR TYN', 'to': 'WPT XIVEP'},
  {'from': 'WPT XIVEP', 'to': 'WPT ANPIG'},
  {'from': 'WPT ANPIG', 'to': 'WPT EGEBI'},
  {'from': 'WPT EGEBI', 'to': 'NDB SQ'},
  {'from': 'NDB SQ', 'to': 'WPT PADNO'},
  {'from': 'WPT PADNO', 'to': 'WPT TAMIX'},
  {'from': 'WPT TAMIX', 'to': 'WPT NOPIN'},
  {'from': 'WPT NOPIN', 'to': 'WPT DUBAG'},
  {'from': 'WPT DUBAG', 'to': 'VOR CGO'},
  {'from': 'VOR CGO', 'to': 'WPT KAMDA'},
  {'from': 'WPT KAMDA', 'to': 'WPT OBLIK'},
  {'from': 'WPT OBLIK', 'to': 'WPT ESDOS'},
  {'from': 'WPT ESDOS', 'to': 'VOR HOK'},
  {'from': 'VOR HOK', 'to': 'VOR LKO'},
  {'from': '

In [25]:
points

{'WPT 0000E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E000000'},
 'WPT 0001E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E001000'},
 'WPT 0002E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E002000'},
 'WPT 0002N': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'W002000'},
 'WPT 0003E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E003000'},
 'WPT 0004E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E004000'},
 'WPT 0004N': {'area': 'GOOO', 'lat': 'N00000', 'lon': 'W004000'},
 'WPT 0005E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E005000'},
 'WPT 0006E': {'area': 'DGAC', 'lat': 'N00000', 'lon': 'E006000'},
 'WPT 0007E': {'area': 'FCCC', 'lat': 'N00000', 'lon': 'E007000'},
 'WPT 0008E': {'area': 'FCCC', 'lat': 'N00000', 'lon': 'E008000'},
 'WPT 0008N': {'area': 'GOOO', 'lat': 'N00000', 'lon': 'W008000'},
 'WPT 0009E': {'area': 'FCCC', 'lat': 'N00000', 'lon': 'E009000'},
 'WPT 0009N': {'area': 'GOOO', 'lat': 'N00000', 'lon': 'W009000'},
 'WPT 0010E': {'area': 'FCCC', 'lat': 'N00000', 'lon': 'E01000